In [16]:
from enum import Enum
from typing import TypedDict, Annotated, List, Dict, Any, Optional
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langgraph.graph import add_messages
from langchain_community.utilities import GoogleSerperAPIWrapper

In [12]:

llm = ChatOpenAI(model="gpt-5-mini")
serper = GoogleSerperAPIWrapper()
tool_web_search = Tool(
    name="web_search",
    func=serper.run,
    description="Useful for when you need more information from the web about a query"
)
tools = [tool_web_search]
llm_with_tools = llm.bind_tools(tools)


In [13]:
class Phase(str, Enum):
    GENERATE = "generate"
    VALIDATE = "validate"
    AUDIT = "audit"
    COMPLETE = "complete"

In [17]:
class State(TypedDict):
    messages: Annotated[List, add_messages]
    user_problem: str
    current_phase: Phase
    iteration_count: int
    
    proposed_architecture: Dict[str, Any]
    architecture_components: Dict[str, Dict[str, Any]]

    validation_feedback: List[Dict[str, Any]]
    audit_feedback: List[Dict[str, Any]]

    active_agents: List[str]
    completed_agents: List[str]

    factual_errors_exist: bool
    design_flaws_exist: bool

    final_architecture: Optional[Dict[str, Any]]
    architecture_summary: Optional[str]
    

In [ ]:
def architect_supervisor(state: State) -> State:
    """
    This function orchestrates the architecture generation process. And Supervisor that coordinates the architect agents.
    """

    system_prompt = f"""
    You are a architect supervisor for AWS cloud architecture.
    Your role is to decompose the user's problem into smaller components on the basis of domain and assign them to different architect domain agents respectively.
    You will also be responsible for coordinating the work of these agents, ensuring that the overall architecture is aligned with the user's requirements and constraints.

    You will be given a user problem and a list of architect domain agents.
    You need to decide which agents are best suited to work on which components of the problem.
    
    User Problem: {state["user_problem"]}
    Current Iteration: {state["iteration_count"]}

    Architect Domain Agents: 
    
    """